In [1]:
from pyspark.sql import SparkSession
from delta import *

builder = (
    SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/home/nil/anaconda3/envs/spark/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nil/.ivy2/cache
The jars for the packages stored in: /home/nil/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5c17cb4f-8906-42d8-870b-69701640bd44;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 224ms :: artifacts dl 10ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |  

- Vacuum: To clean up absolute file

In [2]:
spark.sql(
    """CREATE OR REPLACE TABLE scd2Demo(
    pk1 INT,
    pk2 STRING,
    dim1 INT,
    dim2 INT,
    dim3 int,
    dim4 int,
    active_status string,
    start_date timestamp,
    end_date timestamp
)
USING DELTA
LOCATION '/mnt/sda3/iNeuron/Data Engineering/pySpark/Delta_Lake/delta_lake_06'"""
)

24/02/22 12:32:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/02/22 12:32:56 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


DataFrame[]

In [3]:
spark.sql(
    """INSERT INTO scd2Demo VALUES 
    (111,'Unit1',200,500,800,400,'Y',current_timestamp(),'9999-12-31'),
    (222,'Unit2',900,Null,700,100,'Y',current_timestamp(),'9999-12-31'),
    (333,'Unit3',300,900,250,650,'Y',current_timestamp(),'9999-12-31')
    """
)

DataFrame[]

In [4]:
spark.sql(
    """INSERT INTO scd2Demo VALUES 
    (666,'Unit6',200,500,800,400,'Y',current_timestamp(),'9999-12-31'),
    (777,'Unit7',900,Null,700,100,'Y',current_timestamp(),'9999-12-31'),
    (888,'Unit8',300,900,250,650,'Y',current_timestamp(),'9999-12-31')
    """
)

DataFrame[]

In [5]:
spark.sql("DELETE FROM scd2demo where pk1=777")

DataFrame[num_affected_rows: bigint]

In [6]:
spark.sql("UPDATE scd2Demo SET dim1 = 100 where pk1=666")

DataFrame[num_affected_rows: bigint]

In [7]:
spark.sql("OPTIMIZE scd2Demo").show(truncate=False)

+------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------+
|path                                                                    |metrics                                                                                                                                       |
+------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------+
|file:/mnt/sda3/iNeuron/Data Engineering/pySpark/Delta_Lake/delta_lake_06|{1, 5, {2583, 2583, 2583.0, 1, 2583}, {2367, 2403, 2395.6, 5, 11978}, 1, NULL, 1, 5, 0, false, 0, 0, 1708585398182, 0, 8, 0, NULL, NULL, 9, 9}|
+------------------------------------------------------------------------+------------------------------------------------------

In [8]:
spark.sql("DESCRIBE HISTORY scd2Demo").show(truncate=False)

+-------+-----------------------+------+--------+-----------------------+------------------------------------------------------------------------------+----+--------+---------+-----------+-----------------+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation              |operationParameters                                                           |job |notebook|clusterId|readVersion|isolationLevel   |isBlindAppend|operationMetrics                                                                                                                                                                               

In [9]:
spark.sql("VACUUM scd2Demo DRY RUN").show(truncate=False)

Found 0 files (0 bytes) and directories in a total of 1 directories that are safe to delete.


+----+
|path|
+----+
+----+



In [10]:
# change the duration default is 168 hrs

spark.sql("VACUUM scd2Demo RETAIN 720 HOURS DRY RUN").show(truncate=False)

Found 0 files (0 bytes) and directories in a total of 1 directories that are safe to delete.


+----+
|path|
+----+
+----+

